In [1]:
# 패키지 설치 (Colab 환경에는 패키지가 설치되어 있지 않음)
!pip install causalml

In [2]:
# 필요한 패키지 로드
import causalml
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from causalml.inference.meta import BaseSLearner, BaseTLearner, BaseXLearner, BaseRLearner, BaseDRLearner

# 데이터셋 로드
from causalml.dataset import synthetic_data

In [3]:
np.random.seed(41)
# 사용 방법
num_features = 5
Y, X, T, tau, b, e = synthetic_data(mode=1, n=100, p=num_features)

# S-Learner

- 하나의 모델만 학습함
- 입력 변수 `X`와 함께 **처리 변수 `T` (0 or 1)**를 함께 넣어 학습
- 처리 유무에 따른 효과를 **하나의 모델에서 직접 추정**
- 장점: 간단하고 구현이 쉬움
- 단점: T=0/1에 대한 구분이 잘 안 될 수 있어, 비선형 모델이 필요할 수 있음

**CATE 추정 수식:**
$$
\widehat{\tau}(x) = \hat{f}(x, T=1) - \hat{f}(x, T=0)
$$



In [4]:
# 실제로 S-learner에 활용해보기
model=LinearRegression()

# X와 T를 합쳐서 학습
X_with_T = np.hstack([X, T.reshape(-1, 1)])
model.fit(X_with_T,Y)

# T=0일 때의 예측
X_t0 = np.hstack([X, np.zeros((X.shape[0], 1))])
mu_0 = model.predict(X_t0)

# T=1일 때의 예측
X_t1 = np.hstack([X, np.ones((X.shape[0], 1))])
mu_1 = model.predict(X_t1)
ATE_manual_S = mu_1-mu_0


Lrt = BaseSLearner(learner=LinearRegression())
result = Lrt.estimate_ate(X,T,Y)

# 결과 확인
print('S Learner (ATE) by maual: ', round(ATE_manual_S[0],4))
print('S Learner (ATE) by package: ', round(result[0],4))

S Learner (ATE) by maual:  0.6731
S Learner (ATE) by package:  0.6731


# T-Learner

- **처리군(T=1)**과 **통제군(T=0)** 각각 따로 모델을 학습함 (총 2개)
- 두 모델의 예측값 차이를 통해 CATE 추정
- 장점: 두 집단의 데이터 분포 차이를 잘 반영할 수 있음
- 단점: 각 군의 샘플 수가 적으면 불안정할 수 있음

**CATE 추정 수식:**
$$
\widehat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x)
$$

- $\hat{\mu}_1(x)$: T=1 집단에서 학습한 모델의 예측값  
- $\hat{\mu}_0(x)$: T=0 집단에서 학습한 모델의 예측값

In [5]:
from xgboost import XGBRegressor


# 통제군
model_0 = XGBRegressor()
X_0 = X[T == 0]
Y_0 = Y[T == 0]
model_0.fit(X_0, Y_0)
mu_0=np.mean(model_0.predict(X))


# 실험군
model_1 = LinearRegression()
X_1 = X[T == 1]
Y_1 = Y[T == 1]
model_1.fit(X_1, Y_1)
mu_1 = np.mean(model_1.predict(X))

# ATE 추정
ATE_manual_T = mu_1-mu_0


Lrt = BaseTLearner(learner=LinearRegression(),control_learner=XGBRegressor())
ate,ate_lower,ate_upper = Lrt.estimate_ate(X,T,Y)


# 결과 확인
print('T Learner (ATE) by maual: ', round(ATE_manual_T,4))
print('T Learner (ATE) by package: ', round(ate[0],4))

T Learner (ATE) by maual:  0.5401
T Learner (ATE) by package:  0.5401
